In [ ]:
from cherab.core.math import Interpolate1DCubic, Interpolate2DCubic, IsoMapper2D, AxisymmetricMapper, VectorAxisymmetricMapper
from cherab.compass.equilibrium.equilibrium import COMPASSEquilibrium
from raysect.optical import World
from cherab.openadas import OpenADAS
from cherab.core import Plasma, Species, Maxwellian
from cherab.core.atomic import elements, Line, deuterium, carbon, boron, helium
from cherab.core.model import ExcitationLine, RecombinationLine

from raysect.primitive import Cylinder
from raysect.core import translate, rotate_basis, Vector3D
from raysect.optical.observer import FibreOptic

from scipy.constants import atomic_mass, electron_mass
import numpy as np
from os.path import expanduser""
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
boron_balance = np.loadtxt("../data/boron_balance.txt", skiprows=1)
carbon_balance = np.loadtxt("../data/carbon_balance.txt", skiprows=1)
helium_balance = np.loadtxt("../data/helium_balance.txt", skiprows=1)
deuterium_balance = np.loadtxt("../data/deuterium_balance.txt", skiprows=1)

In [ ]:
def doubleparabola(r, Centre, Edge, p, q):
        return (Centre - Edge) * np.power((1 - np.power((r - r.min()) / (r.max() - r.min()), p)), q) + Edge

In [ ]:
figprfs, ax = plt.subplots(figsize = (8,4))
ax.plot(deuterium_balance[:,0], deuterium_balance[:,3::])
#ax.set_xlim((0,100))

figprfs, ax = plt.subplots(figsize = (8,4))
ax.plot(helium_balance[:,0],helium_balance[:,3::])
#ax.set_xlim((0,100))

figprfs, ax = plt.subplots(figsize = (8,4))
ax.plot(carbon_balance[:,1],carbon_balance[:,3::])
ax.set_xlim((0,200))

figprfs, ax = plt.subplots(figsize = (8,4))
ax.plot(boron_balance[:,1],boron_balance[:,3::])
ax.set_xlim((0,400))

In [ ]:
path = "../data/efit_17636.h5" # path to the efit file
shot_time=1.135
equilibrium = COMPASSEquilibrium(path=path)
equilibrium_slice = equilibrium.time(shot_time)

In [ ]:
world = World()

adas = OpenADAS(permit_extrapolation=True)

plasma = Plasma(parent=world)
plasma.geometry = Cylinder(0.8, 0.8,parent = world,transform = translate(0, 0, -0.4))
plasma.atomic_data = adas

In [ ]:
psin_1d = deuterium_balance[:, 0]

vi_profile = doubleparabola(psin_1d, 6e4, 0, 2, 2) # 1d velocity profile
vi_3d = equilibrium_slice.map3d((psin_1d, vi_profile))

#function returning 3d velocity vector for cherab
flow_velocity = lambda x, y, z: Vector3D(y * vi_3d(x, y, z), - x * vi_3d(x, y, z), 0.) \
/ np.sqrt(x*x + y*y)

#ions
ti_1d = doubleparabola(psin_1d, 500, 0, 2, 2)
ti_3d = equilibrium_slice.map3d((psin_1d, ti_1d))

In [ ]:
#setup electrons
te_1d = deuterium_balance[:,1]
ne_1d = deuterium_balance[:,2]

te_3d = equilibrium_slice.map3d((psin_1d, te_1d))
ne_3d = equilibrium_slice.map3d((psin_1d, ne_1d))
plasma.electron_distribution =  Maxwellian(ne_3d, te_3d, flow_velocity, electron_mass)

In [ ]:
#we have to do the hard way, otherwise anuthing outside lcfs is ignored.
#todo: use blending?
d0n_1d = Interpolate1DCubic(psin_1d, deuterium_balance[:,3] * deuterium_balance[:,2], extrapolate=True) #interpolate profile
d0n_2d = IsoMapper2D(equilibrium_slice.psi_normalised, d0n_1d) #map to 2D
d0n_3d = AxisymmetricMapper(d0n_2d) #map to 3D
d0_distribution = Maxwellian(d0n_3d, ti_3d, flow_velocity, deuterium.atomic_weight * atomic_mass)
d0_species = Species(deuterium, 0, d0_distribution)

d1n_1d = Interpolate1DCubic(psin_1d, deuterium_balance[:,4] * deuterium_balance[:,4], extrapolate=True) #interpolate profile
d1n_2d = IsoMapper2D(equilibrium_slice.psi_normalised, d1n_1d) #map to 2D
d1n_3d = AxisymmetricMapper(d1n_2d) #map to 3D
d1_distribution = Maxwellian(d1n_3d, ti_3d, flow_velocity, deuterium.atomic_weight * atomic_mass)
d1_species = Species(deuterium, 1, d1_distribution)


In [ ]:
#too: use mappers
helium_3d = []
helium_distribution = []
helium_species = []
helium_fraction = 0.1
for i in range(3):
    d1 = Interpolate1DCubic(helium_balance[:,0], helium_balance[:,i+3] * helium_balance[:,2]* helium_fraction , extrapolate=True)
    d2 = IsoMapper2D(equilibrium_slice.psi_normalised, d1) #map to 2D
    helium_3d.append(AxisymmetricMapper(d2)) #map to 3D
    helium_distribution.append(Maxwellian(helium_3d[-1], ti_3d, flow_velocity,
                                          helium.atomic_weight * atomic_mass))
    helium_species.append(Species(helium, i, helium_distribution[-1]))
    
carbon_3d = []
carbon_distribution = []
carbon_species = []
carbon_fraction = 0.03
for i in range(7):
    d1 = Interpolate1DCubic(carbon_balance[:,0], carbon_balance[:,i+3] * carbon_balance[:,2] * carbon_fraction, extrapolate=True)
    d2 = IsoMapper2D(equilibrium_slice.psi_normalised, d1) #map to 2D
    carbon_3d.append(AxisymmetricMapper(d2)) #map to 3D
    carbon_distribution.append(Maxwellian(carbon_3d[-1], ti_3d, flow_velocity,
                                          carbon.atomic_weight * atomic_mass))
    carbon_species.append(Species(carbon, i, carbon_distribution[-1]))
    
boron_3d = []
boron_distribution = []
boron_species = []
boron_fraction = 0.01
for i in range(5):
    d1 = Interpolate1DCubic(carbon_balance[:,0], boron_balance[:,i+3] * boron_balance[:,2] * boron_fraction, extrapolate=True)
    d2 = IsoMapper2D(equilibrium_slice.psi_normalised, d1) #map to 2D
    boron_3d.append(AxisymmetricMapper(d2)) #map to 3D
    boron_distribution.append(Maxwellian(boron_3d[-1], ti_3d, flow_velocity,
                                          boron.atomic_weight * atomic_mass))
    boron_species.append(Species(boron, i, boron_distribution[-1]))

In [ ]:
xrange = np.arange(equilibrium_slice.r_data.min(), equilibrium_slice.r_data.max(), 0.005)
yrange = np.arange(equilibrium_slice.z_data.min(), equilibrium_slice.z_data.max(), 0.005)
psi_test = np.zeros((yrange.shape[0], xrange.shape[0]))
c6n_test = np.zeros((yrange.shape[0], xrange.shape[0]))
c5n_test = np.zeros((yrange.shape[0], xrange.shape[0]))
c4n_test = np.zeros((yrange.shape[0], xrange.shape[0]))

for i, x in enumerate(xrange):
    for j, y in enumerate(yrange):
        psi_test[j, i] = equilibrium_slice.psi_normalised(x, y)
        c4n_test[j, i] = carbon_3d[-3](x, 0, y)
        c5n_test[j, i] = carbon_3d[-2](x, 0, y)
        c6n_test[j, i] = carbon_3d[-1](x, 0, y)

In [ ]:
fig_profiles, ax = plt.subplots(1, 3)
ax[0].contourf(xrange, yrange, c4n_test,20)
ax[0].contour(xrange, yrange, psi_test,10, colors="w")
ax[0].plot(equilibrium_slice.lcfs_polygon[:,0], equilibrium_slice.lcfs_polygon[:,1],"-C1")
ax[0].set_title("C4+")
ax[0].set_aspect(1)
ax[0].plot(equilibrium_slice.limiter_polygon[:,0], equilibrium_slice.limiter_polygon[:,1],"-r")

ax[1].contourf(xrange, yrange, c5n_test,20)
ax[1].contour(xrange, yrange, psi_test,10, colors="w")
ax[1].plot(equilibrium_slice.lcfs_polygon[:,0], equilibrium_slice.lcfs_polygon[:,1],"-C1")
ax[1].set_title("C5+")
ax[1].set_aspect(1)
ax[1].plot(equilibrium_slice.limiter_polygon[:,0], equilibrium_slice.limiter_polygon[:,1],"-r")

ax[2].contourf(xrange, yrange, c6n_test,20)
ax[2].contour(xrange, yrange, psi_test,10, colors="w")
ax[2].plot(equilibrium_slice.lcfs_polygon[:,0], equilibrium_slice.lcfs_polygon[:,1],"-C1")
ax[2].set_title("C6+")
ax[2].set_aspect(1)
ax[2].plot(equilibrium_slice.limiter_polygon[:,0], equilibrium_slice.limiter_polygon[:,1],"-r")

In [ ]:
plasma.electron_distribution =  Maxwellian(ne_3d, te_3d, flow_velocity, electron_mass)
plasma.composition = [d0_species, d1_species] + helium_species + carbon_species + boron_species
plasma.b_field = VectorAxisymmetricMapper(equilibrium_slice.b_field)

In [ ]:
d_alpha = Line(deuterium, 0, (3, 2))
#line_boron= Line(boron, 4, (7, 6))
#line_carbon = Line(carbon, 5, (8, 7))
#line_carbon = Line(carbon, 2,)

In [ ]:
plasma.models = [
    ExcitationLine(d_alpha)
]

In [ ]:
fibre = FibreOptic(acceptance_angle=1, radius=0.001, parent=world,
                   transform=translate(0.56,0,0) *
                             rotate_basis(Vector3D(-1, 0, 0), Vector3D(0, 0, 1)))
fibre.min_wavelength = 490
fibre.max_wavelength = 670
fibre.spectral_bins = 2000
fibre.pixel_samples = 2000



In [ ]:
plt.ion()
fibre.observe()
